In [1]:
import requests
s = requests.get("https://raw.githubusercontent.com/Niraj-Kamdar/IT413-NoSQL/master/Lab01/problem-2/input/1901")
dbutils.fs.mkdirs("dbfs:/lab2_3_2")
with open("/dbfs/lab2_3_2/1901", "w") as f:
    f.write(s.text)
s = requests.get("https://raw.githubusercontent.com/Niraj-Kamdar/IT413-NoSQL/master/Lab01/problem-2/input/1902")
with open("/dbfs/lab2_3_2/1902", "w") as f:
    f.write(s.text)

In [2]:
f1 = sc.textFile("dbfs:/lab2_3_2/1901")
f2 = sc.textFile("dbfs:/lab2_3_2/1902")
MISSING = 9999
m1 = f1.filter(lambda x: x[92:93] in "01459" and abs(int(x[87:92])) != MISSING).map(lambda x: (x[15:19], int(x[87:92]))).reduceByKey(lambda x, y: x if x > y else y)
print(m1.take(1))
m2 = f2.filter(lambda x: x[92:93] in "01459" and abs(int(x[87:92])) != MISSING).map(lambda x: (x[15:19], int(x[87:92]))).reduceByKey(lambda x, y: x if x > y else y)
print(m2.take(1))

[('1901', 317)]
[('1902', 244)]

In [3]:
import requests
s = requests.get("https://raw.githubusercontent.com/Niraj-Kamdar/IT413-NoSQL/master/Lab01/problem-3/input/web_access_log.txt")
dbutils.fs.mkdirs("dbfs:/lab2_3_2")
with open("/dbfs/lab2_3_2/web_access_log", "w") as f:
    f.write(s.text)


In [4]:
import re
rdd = sc.textFile("dbfs:/lab2_3_2/web_access_log")
match_ext = {".png", ".jpg", ".gif"}
res = rdd.filter(lambda x: re.search(r"GET ([^=]+.[a-zA-Z]{3}) HTTP",x)).map(lambda x: re.search(r"GET ([^=]+.[a-zA-Z]{3}) HTTP",x).groups()[0].lower()).map(lambda x: [(i, 1) for i in match_ext if i in x]).filter(lambda x: x != []).map(lambda x: x[0]).reduceByKey(lambda x, y: x+y)
print(res.take(5))

[('.jpg', 20063), ('.png', 1571), ('.gif', 1267)]

In [5]:
pattern = r"\[(\d{2})/([A-Z]{1}[a-z]{2})/(\d{4}):(\d{2}):(\d{2}):(\d{2}) [+-]([\d]{4})\][^[\]]+(\d{3}) (\d+) \""
res = rdd.filter(lambda x: re.search(pattern, x)).map(lambda x: re.search(pattern, x).groups()).map(lambda x: (f"{x[1]}-{x[2]}", (1, int(x[8])))).reduceByKey(lambda x, y: (x[0]+y[0], x[1]+y[1]))
print(res.take(5))

[('Feb-2016', (64113, 1412992330)), ('Dec-2015', (13669, 286875357)), ('Jan-2016', (28065, 1840852372))]

In [6]:
pattern = r"\[((\d{2})/([A-Z]{1}[a-z]{2})/(\d{4})):(\d{2}):(\d{2}):(\d{2}) [+-]([\d]{4})\][^[\]]+(\d{3}) (\d+) \"(http[s]*://[^\"]+)\""
res = rdd.filter(lambda x: re.search(pattern, x)).map(lambda x: re.search(pattern, x).groups()).filter(lambda x: x[8] == "404").map(lambda x: (x[0], x[10]))
print(res.take(10))

[('12/Dec/2015', 'http://almhuette-raith.at/'), ('12/Dec/2015', 'http://www.almhuette-raith.at/'), ('13/Dec/2015', 'http://www.almhuette-raith.at/'), ('13/Dec/2015', 'http://www.almhuette-raith.at/'), ('13/Dec/2015', 'http://www.almhuette-raith.at/'), ('13/Dec/2015', 'http://almhuette-raith.at/'), ('13/Dec/2015', 'http://almhuette-raith.at/'), ('13/Dec/2015', 'http://almhuette-raith.at/'), ('13/Dec/2015', 'http://www.almhuette-raith.at/'), ('13/Dec/2015', 'http://www.almhuette-raith.at/')]